In [2]:
import ciropt as co
import cvxpy as cp

In [3]:
import numpy as np

In [8]:
L_smooth_h = 0.1 #np.inf
L_smooth_f = 10
# mu = 1 #0.01
mu = 0 #0.01

# h = 1/2
R = 1

# W = np.array([
#     [1/6, 1/2, 1/3],
#     [1/2, 1/4, 1/4],
#     [1/3, 1/4, 5/12]
# ])

W = np.array([
    [1/3, 2/3, 0],
    [2/3, 1/6, 1/6],
    [0, 1/6, 5/6]
])

# W12, W13, W23 = 2/3, 0, 1/6
# R12, R13, R13 = R * W[0][1], R * W[0][2], R * W[1][2]


solver = "ipopt"
# solver = "ipopt_qcqp"
# solver = "ipopt_qcqp_matrix"
# solver = "cvx_dccp"
# solver = "qcqp_sni"
# solver = "bisection_b"

# Ciropt problem

In [9]:
problem = co.pg_extra_line3(mu, L_smooth_h, L_smooth_f, R, W)
problem.obj = problem.b + problem.d /2
# problem.obj = problem.b

bounds = {  
            'd': {"ub": 100, "lb": 1e-4},
            'b': {"ub": 100, "lb": 1e-4},
            'h': {"ub": 100, "lb": 1e-4},
            'gamma': {"ub": 100, "lb": 1e-4},
            'alpha': {"ub": 1, "lb": 1},
            'beta': {"ub": 0, "lb": 0},
            }

res, sol = problem.solve(solver=solver, bounds=bounds, verbose=True)[:2]

vars = problem.vars
print(res['b'])

dim_G=20, dim_F=15
Ipopt total # of variables = 448
Actual # of variables = 257
This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:    17492
Number of nonzeros in inequality constraint Jacobian.:       74
Number of nonzeros in Lagrangian Hessian.............:     1541

Total number of variables............................:      446
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      416
Total number of inequality constraints...............:       74
        inequality constraints with only lower bounds:       68
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        6

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  

In [10]:
res

{'b': 0.5470025824558842,
 'h': 0.5470025928631261,
 'd': 0.3952161905003262,
 'alpha': 1.0,
 'beta': 6.217248937642383e-15,
 'gamma': 0.2734993082134527}

# PEP verification

In [11]:
problem = co.pg_extra_line3(mu, L_smooth_h, L_smooth_f, R, W, params=res)
diff = problem.solve(verbose=0, solver=cp.MOSEK)
print(f"{diff=}")

diff=-3.384883442980602e-09
